In [2]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [3]:
# read in with pandas
winter1 = pd.read_csv ('fok20-21data.txt')
winter2 = pd.read_csv ('fok21-22data.txt')
winter3 = pd.read_csv ('fok22-23data.txt')

In [4]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [5]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [6]:
precip_days2

,station,datetime,precip(mm),wxcodes
30,FOK,2021-12-02 06:53:00,0.51,-RA
51,FOK,2021-12-03 03:43:00,0.51,-RA
52,FOK,2021-12-03 03:53:00,0.51,-RA
129,FOK,2021-12-06 07:32:00,0.25,-RA
131,FOK,2021-12-06 08:46:00,T,VCTS -RA
...,...,...,...,...
2749,FOK,2022-02-25 14:51:00,2.79,RA BR
2750,FOK,2022-02-25 14:53:00,2.79,RA BR
2751,FOK,2022-02-25 15:07:00,0.76,RA BR
2752,FOK,2022-02-25 15:53:00,2.54,-RA


In [7]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [8]:
precip_days3[660:700]

,station,datetime,precip(mm),wxcodes


In [9]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [10]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [11]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [12]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [13]:
fok = LocationInfo(
    'Westhampton Beach', 'New York', 
    latitude= 40.8094,
    longitude= -72.6458, 
    timezone='America/New_York'
)

fok

LocationInfo(name='Westhampton Beach', region='New York', timezone='America/New_York', latitude=40.8094, longitude=-72.6458)

In [14]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(fok.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [15]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(fok.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [16]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(fok.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [17]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:56:18  2020-12-01 16:23:10  2020-12-01
1   2020-12-02 06:57:18  2020-12-02 16:22:57  2020-12-02
2   2020-12-03 06:58:18  2020-12-03 16:22:47  2020-12-03
3   2020-12-04 06:59:16  2020-12-04 16:22:38  2020-12-04
4   2020-12-05 07:00:13  2020-12-05 16:22:32  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:31:56  2021-02-24 17:36:05  2021-02-24
86  2021-02-25 06:30:27  2021-02-25 17:37:15  2021-02-25
87  2021-02-26 06:28:57  2021-02-26 17:38:24  2021-02-26
88  2021-02-27 06:27:27  2021-02-27 17:39:34  2021-02-27
89  2021-02-28 06:25:56  2021-02-28 17:40:42  2021-02-28

[90 rows x 3 columns]


In [18]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:56:03  2021-12-01 16:23:14  2021-12-01
1   2021-12-02 06:57:04  2021-12-02 16:23:00  2021-12-02
2   2021-12-03 06:58:03  2021-12-03 16:22:49  2021-12-03
3   2021-12-04 06:59:02  2021-12-04 16:22:40  2021-12-04
4   2021-12-05 06:59:59  2021-12-05 16:22:33  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:32:17  2022-02-24 17:35:48  2022-02-24
86  2022-02-25 06:30:48  2022-02-25 17:36:58  2022-02-25
87  2022-02-26 06:29:19  2022-02-26 17:38:08  2022-02-26
88  2022-02-27 06:27:49  2022-02-27 17:39:17  2022-02-27
89  2022-02-28 06:26:18  2022-02-28 17:40:26  2022-02-28

[90 rows x 3 columns]


In [19]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:55:48  2022-12-01 16:23:17  2022-12-01
1   2022-12-02 06:56:49  2022-12-02 16:23:03  2022-12-02
2   2022-12-03 06:57:49  2022-12-03 16:22:51  2022-12-03
3   2022-12-04 06:58:48  2022-12-04 16:22:41  2022-12-04
4   2022-12-05 06:59:45  2022-12-05 16:22:34  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:32:38  2023-02-24 17:35:31  2023-02-24
86  2023-02-25 06:31:10  2023-02-25 17:36:41  2023-02-25
87  2023-02-26 06:29:41  2023-02-26 17:37:51  2023-02-26
88  2023-02-27 06:28:11  2023-02-27 17:39:00  2023-02-27
89  2023-02-28 06:26:40  2023-02-28 17:40:09  2023-02-28

[90 rows x 3 columns]


In [20]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [21]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FOK,2020-12-01 00:53:00,1.0200,-RA BR,2020-12-01,2020-12-01 06:56:18,2020-12-01 16:23:10
1,FOK,2020-12-01 02:53:00,0.5100,-RA BR,2020-12-01,2020-12-01 06:56:18,2020-12-01 16:23:10
2,FOK,2020-12-04 20:53:00,0.2500,-RA,2020-12-04,2020-12-04 06:59:16,2020-12-04 16:22:38
3,FOK,2020-12-04 21:53:00,0.5100,-RA,2020-12-04,2020-12-04 06:59:16,2020-12-04 16:22:38
4,FOK,2020-12-04 23:53:00,0.0001,-RA,2020-12-04,2020-12-04 06:59:16,2020-12-04 16:22:38
...,...,...,...,...,...,...,...
247,FOK,2021-02-27 13:53:00,1.0200,-RA BR,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34
248,FOK,2021-02-27 14:53:00,1.2700,-RA BR,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34
249,FOK,2021-02-27 15:53:00,2.2900,-RA BR,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34
250,FOK,2021-02-27 17:53:00,0.5100,-RA BR,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34


In [22]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FOK,2021-12-02 06:53:00,0.5100,-RA,2021-12-02,2021-12-02 06:57:04,2021-12-02 16:23:00
1,FOK,2021-12-03 03:53:00,0.5100,-RA,2021-12-03,2021-12-03 06:58:03,2021-12-03 16:22:49
2,FOK,2021-12-06 08:53:00,0.2500,VCTS -RA,2021-12-06,2021-12-06 07:00:55,2021-12-06 16:22:28
3,FOK,2021-12-06 09:53:00,4.5700,-RA,2021-12-06,2021-12-06 07:00:55,2021-12-06 16:22:28
4,FOK,2021-12-07 02:53:00,0.2500,-RA,2021-12-07,2021-12-07 07:01:50,2021-12-07 16:22:26
...,...,...,...,...,...,...,...
238,FOK,2022-02-25 12:53:00,0.5100,-RA BR,2022-02-25,2022-02-25 06:30:48,2022-02-25 17:36:58
239,FOK,2022-02-25 13:53:00,3.0500,-RA BR,2022-02-25,2022-02-25 06:30:48,2022-02-25 17:36:58
240,FOK,2022-02-25 14:53:00,2.7900,RA BR,2022-02-25,2022-02-25 06:30:48,2022-02-25 17:36:58
241,FOK,2022-02-25 15:53:00,2.5400,-RA,2022-02-25,2022-02-25 06:30:48,2022-02-25 17:36:58


In [23]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FOK,2022-12-01 00:53:00,0.0001,-RA,2022-12-01,2022-12-01 06:55:48,2022-12-01 16:23:17
1,FOK,2022-12-03 13:53:00,0.0001,-RA,2022-12-03,2022-12-03 06:57:49,2022-12-03 16:22:51
2,FOK,2022-12-03 14:53:00,0.7600,-RA,2022-12-03,2022-12-03 06:57:49,2022-12-03 16:22:51
3,FOK,2022-12-03 15:53:00,0.7600,-RA BR,2022-12-03,2022-12-03 06:57:49,2022-12-03 16:22:51
4,FOK,2022-12-03 16:53:00,1.5200,-RA BR,2022-12-03,2022-12-03 06:57:49,2022-12-03 16:22:51
...,...,...,...,...,...,...,...
222,FOK,2023-02-22 22:53:00,0.7600,-RA,2023-02-22,2023-02-22 06:35:32,2023-02-22 17:33:10
223,FOK,2023-02-23 01:53:00,0.2500,-RA BR,2023-02-23,2023-02-23 06:34:06,2023-02-23 17:34:21
224,FOK,2023-02-23 02:53:00,0.0001,-RA,2023-02-23,2023-02-23 06:34:06,2023-02-23 17:34:21
225,FOK,2023-02-23 03:53:00,0.2500,-RA BR,2023-02-23,2023-02-23 06:34:06,2023-02-23 17:34:21


In [24]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
7,FOK,2020-12-05 07:53:00,0.51,-RA,2020-12-05,2020-12-05 07:00:13,2020-12-05 16:22:32
8,FOK,2020-12-05 08:53:00,1.27,RA,2020-12-05,2020-12-05 07:00:13,2020-12-05 16:22:32
9,FOK,2020-12-05 09:53:00,2.79,RA BR,2020-12-05,2020-12-05 07:00:13,2020-12-05 16:22:32
10,FOK,2020-12-05 10:53:00,3.81,-RA,2020-12-05,2020-12-05 07:00:13,2020-12-05 16:22:32
11,FOK,2020-12-05 11:53:00,1.27,RA BR,2020-12-05,2020-12-05 07:00:13,2020-12-05 16:22:32
...,...,...,...,...,...,...,...
245,FOK,2021-02-27 11:53:00,0.25,-RA,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34
246,FOK,2021-02-27 12:53:00,0.25,-RA BR,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34
247,FOK,2021-02-27 13:53:00,1.02,-RA BR,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34
248,FOK,2021-02-27 14:53:00,1.27,-RA BR,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34


In [25]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [26]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
8,FOK,2020-12-05 08:53:00,1.27,RA,2020-12-05,2020-12-05 07:00:13,2020-12-05 16:22:32,2020-12-05 08:00:13,2020-12-05 15:22:32
9,FOK,2020-12-05 09:53:00,2.79,RA BR,2020-12-05,2020-12-05 07:00:13,2020-12-05 16:22:32,2020-12-05 08:00:13,2020-12-05 15:22:32
10,FOK,2020-12-05 10:53:00,3.81,-RA,2020-12-05,2020-12-05 07:00:13,2020-12-05 16:22:32,2020-12-05 08:00:13,2020-12-05 15:22:32
11,FOK,2020-12-05 11:53:00,1.27,RA BR,2020-12-05,2020-12-05 07:00:13,2020-12-05 16:22:32,2020-12-05 08:00:13,2020-12-05 15:22:32
12,FOK,2020-12-05 12:53:00,7.87,RA BR,2020-12-05,2020-12-05 07:00:13,2020-12-05 16:22:32,2020-12-05 08:00:13,2020-12-05 15:22:32
...,...,...,...,...,...,...,...,...,...
245,FOK,2021-02-27 11:53:00,0.25,-RA,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34,2021-02-27 07:27:27,2021-02-27 16:39:34
246,FOK,2021-02-27 12:53:00,0.25,-RA BR,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34,2021-02-27 07:27:27,2021-02-27 16:39:34
247,FOK,2021-02-27 13:53:00,1.02,-RA BR,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34,2021-02-27 07:27:27,2021-02-27 16:39:34
248,FOK,2021-02-27 14:53:00,1.27,-RA BR,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34,2021-02-27 07:27:27,2021-02-27 16:39:34


In [27]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [28]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [29]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 1.5720505617977525
2021-2022: 0.7848166666666666
2022-2023: 0.814382258064516


In [30]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FOK,2020-12-01 00:53:00,1.0200,-RA BR,2020-12-01,2020-12-01 06:56:18,2020-12-01 16:23:10
1,FOK,2020-12-01 02:53:00,0.5100,-RA BR,2020-12-01,2020-12-01 06:56:18,2020-12-01 16:23:10
2,FOK,2020-12-04 20:53:00,0.2500,-RA,2020-12-04,2020-12-04 06:59:16,2020-12-04 16:22:38
3,FOK,2020-12-04 21:53:00,0.5100,-RA,2020-12-04,2020-12-04 06:59:16,2020-12-04 16:22:38
4,FOK,2020-12-04 23:53:00,0.0001,-RA,2020-12-04,2020-12-04 06:59:16,2020-12-04 16:22:38
...,...,...,...,...,...,...,...
241,FOK,2021-02-22 22:53:00,0.7600,-RA,2021-02-22,2021-02-22 06:34:51,2021-02-22 17:33:45
242,FOK,2021-02-22 23:53:00,0.5100,-RA,2021-02-22,2021-02-22 06:34:51,2021-02-22 17:33:45
243,FOK,2021-02-23 18:53:00,0.2500,-RA BR,2021-02-23,2021-02-23 06:33:24,2021-02-23 17:34:55
250,FOK,2021-02-27 17:53:00,0.5100,-RA BR,2021-02-27,2021-02-27 06:27:27,2021-02-27 17:39:34


In [31]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_15044\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [32]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [33]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [34]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.8656661971830985
2021-2022: 0.5285729166666667
2022-2023: 0.9659955223880596


In [35]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('fok.csv', index=False)